In [1]:
import pandas as pd
import numpy as np
import random
import warnings
import random
import time
import transformers
from clean import clean_txt
from sklearn.feature_extraction.text import TfidfVectorizer
from extract_feature import feature_extraction_job, feature_extraction_user
from utils import compute_similarity, get_applicant_id_info, get_recommendation


warnings.filterwarnings("ignore")


c:\Users\84363\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
final_df_jobs = pd.read_csv('./clean_data/final_df_jobs.csv')
final_df_user = pd.read_csv('./clean_data/final_user_df.csv')
combine_df_job = pd.read_csv('./data/Combined_Jobs_Final.csv')
df_job_view = pd.read_csv('./data/Job_Views.csv')
df_experience = pd.read_csv('./data/Experience.csv')
df_interest_position = pd.read_csv('./data/Positions_Of_Interest.csv')

In [23]:
i = random.randint(0, len(final_df_user))
random_applicant_id = final_df_user['Applicant.ID'][i]
feature_text = final_df_user['text'][i]
_ = get_applicant_id_info(random_applicant_id, df_job_view, df_experience, df_interest_position)

Applicant Id: 10250 đã bấm vào tin tức tuyển dụng trên website với             
Vị trí:              
Tên công ty tương ứng là:              
Thành phố tương ứng là:              
Vị trí có kinh nghiệm: Security Officer             
Vị trí mong muốn: security


In [33]:
def get_scores(type):
    start_time = time.time()
    assert type in range(3)
    if type != 2:
        try:
            user_feature_vector = feature_extraction_user([feature_text], final_df_jobs, type)
            job_feature_vector, _ = feature_extraction_job(final_df_jobs, type)
            cosine_scores = compute_similarity(job_feature_vector, user_feature_vector)
            top = sorted(range(len(cosine_scores)), key=lambda i: cosine_scores[i], reverse=True)[:10]
            list_scores = [cosine_scores[i][0][0] for i in top]
            print(f"Time excution: {time.time() - start_time:.6f}s")
            return top, list_scores
        except:
            print(f"Applicant ID: {random_applicant_id} không để lại thông tin")
    else:
        top, list_scores = feature_extraction_user([feature_text], final_df_jobs, type)
        print(f"Time excution: {time.time() - start_time:.6f}s")
        return top[::-1], list_scores[::-1]

In [25]:
top_tfidf, list_scores_tfidf = get_scores(0)

Time excution: 40.829658s


In [26]:
top_bow, list_scores_bow = get_scores(1)

Time excution: 48.566284s


In [34]:
top_knn, list_scores_knn = get_scores(2)

Time excution: 0.792434s


### Top Recommendations sử dụng TF-IDF

In [30]:
get_recommendation(top_tfidf, combine_df_job, final_df_jobs, list_scores_tfidf, applicant_id=random_applicant_id)

,JobID,title,Position,Company,City,Job.Description,Employment.Type,score
0,283989.0,Armed Security Officers @ SOI Security (Securi...,Armed Security Officers,"SOI Security (Security One, Inc.)",Miami,Armed Security Officer \r\n,Full-Time/Part-Time,0.729198
1,288401.0,Hospital Security Officers @ Universal Protect...,Hospital Security Officers,Universal Protection Service,Spokane,Security Officers! Work for the Best at the B...,Part-Time,0.727478
2,294341.0,"Armed Security Officer @ Security One, Inc.",Armed Security Officer,"Security One, Inc.",Belleville,\r\nArmed\r\nSecurity Officers\r\nIL and MO,Full-Time/Part-Time,0.721482
3,272895.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Nashville,ABOUT US:\r\n&nbsp;\r\nU.S. Security Associate...,Full-Time/Part-Time,0.687775
4,267787.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Winter Park,ABOUT US:\r\nU.S. Security Associates (USA) is...,Full-Time/Part-Time,0.682842
5,266810.0,Security Office - Part Time,Security Office - Part Time,NaN,Fort Worth,Security Officer (part time/relief position)\r...,Part-Time,0.675717
6,151131.0,Security Officer @ Confidential,Security Officer,Confidential,Chillicothe,We are looking for a reliable part time Securi...,Part-Time,0.671045
7,273386.0,Corporate Security Officer @ US Security Assoc...,Corporate Security Officer,US Security Associates,Atlanta,&ldquo;Thank you very much for expressing inte...,Full-Time/Part-Time,0.656316
8,307750.0,Corporate Security Officer @ US Security Assoc...,Corporate Security Officer,US Security Associates,Atlanta,&ldquo;Thank you very much for expressing inte...,Full-Time/Part-Time,0.656316
9,303066.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Houston,"Under direct supervision, the unarmed Security...",Full-Time/Part-Time,0.649943


### Top Recommendations sử dụng Bag-of-Word

In [31]:
get_recommendation(top_bow, combine_df_job, final_df_jobs, list_scores_bow, applicant_id=random_applicant_id)

,JobID,title,Position,Company,City,Job.Description,Employment.Type,score
0,283989.0,Armed Security Officers @ SOI Security (Securi...,Armed Security Officers,"SOI Security (Security One, Inc.)",Miami,Armed Security Officer \r\n,Full-Time/Part-Time,0.766965
1,288401.0,Hospital Security Officers @ Universal Protect...,Hospital Security Officers,Universal Protection Service,Spokane,Security Officers! Work for the Best at the B...,Part-Time,0.750757
2,266810.0,Security Office - Part Time,Security Office - Part Time,NaN,Fort Worth,Security Officer (part time/relief position)\r...,Part-Time,0.723546
3,279545.0,Security Officers - DC with Hotel Security exp...,Security Officers - DC with Hotel Security exp.,Triton Security Inc.,Fairfax,&nbsp;\r\n\r\nTriton Security is looking to fi...,Full-Time/Part-Time,0.718411
4,272895.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Nashville,ABOUT US:\r\n&nbsp;\r\nU.S. Security Associate...,Full-Time/Part-Time,0.716737
5,294341.0,"Armed Security Officer @ Security One, Inc.",Armed Security Officer,"Security One, Inc.",Belleville,\r\nArmed\r\nSecurity Officers\r\nIL and MO,Full-Time/Part-Time,0.701646
6,289892.0,Security Officer @ McRoberts Maritime Security,Security Officer,McRoberts Maritime Security,Galveston,McRoberts Maritime Security is hiring for curr...,Part-Time,0.701096
7,267787.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Winter Park,ABOUT US:\r\nU.S. Security Associates (USA) is...,Full-Time/Part-Time,0.698947
8,273386.0,Corporate Security Officer @ US Security Assoc...,Corporate Security Officer,US Security Associates,Atlanta,&ldquo;Thank you very much for expressing inte...,Full-Time/Part-Time,0.687777
9,307750.0,Corporate Security Officer @ US Security Assoc...,Corporate Security Officer,US Security Associates,Atlanta,&ldquo;Thank you very much for expressing inte...,Full-Time/Part-Time,0.687777


### Top Recommendations sử dụng KNN

In [35]:
get_recommendation(top_knn, combine_df_job, final_df_jobs, list_scores_knn, applicant_id=random_applicant_id)

,JobID,title,Position,Company,City,Job.Description,Employment.Type,score
0,303066.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Houston,"Under direct supervision, the unarmed Security...",Full-Time/Part-Time,0.836728
1,307750.0,Corporate Security Officer @ US Security Assoc...,Corporate Security Officer,US Security Associates,Atlanta,&ldquo;Thank you very much for expressing inte...,Full-Time/Part-Time,0.829077
2,273386.0,Corporate Security Officer @ US Security Assoc...,Corporate Security Officer,US Security Associates,Atlanta,&ldquo;Thank you very much for expressing inte...,Full-Time/Part-Time,0.829077
3,151131.0,Security Officer @ Confidential,Security Officer,Confidential,Chillicothe,We are looking for a reliable part time Securi...,Part-Time,0.811116
4,266810.0,Security Office - Part Time,Security Office - Part Time,NaN,Fort Worth,Security Officer (part time/relief position)\r...,Part-Time,0.805335
5,267787.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Winter Park,ABOUT US:\r\nU.S. Security Associates (USA) is...,Full-Time/Part-Time,0.796440
6,272895.0,Unarmed Security Officer @ US Security Associates,Unarmed Security Officer,US Security Associates,Nashville,ABOUT US:\r\n&nbsp;\r\nU.S. Security Associate...,Full-Time/Part-Time,0.790222
7,294341.0,"Armed Security Officer @ Security One, Inc.",Armed Security Officer,"Security One, Inc.",Belleville,\r\nArmed\r\nSecurity Officers\r\nIL and MO,Full-Time/Part-Time,0.746348
8,288401.0,Hospital Security Officers @ Universal Protect...,Hospital Security Officers,Universal Protection Service,Spokane,Security Officers! Work for the Best at the B...,Part-Time,0.738271
9,283989.0,Armed Security Officers @ SOI Security (Securi...,Armed Security Officers,"SOI Security (Security One, Inc.)",Miami,Armed Security Officer \r\n,Full-Time/Part-Time,0.735937
